# 1. Install the Required libraries

In [ ]:
%pip install --upgrade semantic-kernel

# 2. Create your environment variables .env file

Add your environment variables then run the cell to create the *.env* file with your environment variable.

In [ ]:
%%writefile .env
# Environment variables obtained from Azure OpenAI
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=""
AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME=""
AZURE_OPENAI_DEPLOYMENT_NAME=""
AZURE_OPENAI_ENDPOINT=""
AZURE_OPENAI_API_KEY=""
# Environment variable obtained from Azure Cosmos DB for MongoDB vCore
AZCOSMOS_CONNSTR=""
# Environment variables you set to be used by the code
AZCOSMOS_API="mongo-vcore" # currently, semantic kernel only supports vCore
AZCOSMOS_DATABASE_NAME=""
AZCOSMOS_CONTAINER_NAME=""

# 3. Load the environment variables

In [1]:
# load the environment variables file
from dotenv import dotenv_values

config = dotenv_values(".env")

Some of the parameters needed by [Azure Cosmos DB for MongoDB vCore](https://learn.microsoft.com/azure/cosmos-db/mongodb/vcore/vector-search) to create the vector search index are handled by semantic kernel.

In this guide, we are using `text-embedding-ada-002` embedding model to generate the embeddings which uses a 1536-dimensional embedding vector.

The `num_lists` is an integer that represents of clusters that the inverted file (IVF) index uses to group the vector data.

The `similarity` used with IVF index here is the `COS` (cosine distance) but you can also try `L2` (Euclidean distance), and `IP` (inner product). For more information see the [Understand embeddings in Azure OpenAI Service article](https://learn.microsoft.com/azure/ai-services/openai/concepts/understand-embeddings#cosine-similarity).

In [2]:
# collection name will be used multiple times in the code so we store it in a variable
collection_name = config.get("AZCOSMOS_CONTAINER_NAME")

# Vector search index parameters
index_name = "VectorSearchIndex"
vector_dimensions = (
    1536  # text-embedding-ada-002 uses a 1536-dimensional embedding vector
)
num_lists = 1
similarity = "COS"  # cosine distance

# 4. Create Helper Functions

This function takes in a json file of NoSQL records and checks if your data exists in the database using the id of the record then skips the record if it exists or generates embeddings and uploads the database record along with it's embedding.

The `save_information` function does two things: generate embeddings + upload the data to your database.

Learn more about the semantic kernel memory store [here](https://learn.microsoft.com/semantic-kernel/memories/) and the embeddings [here](https://learn.microsoft.com/semantic-kernel/memories/embeddings).

In [3]:
import json


async def upsert_data_to_memory_store(
    kernel_memory_store: callable, memory_store: callable, data_file_path: str
) -> None:
    """
    This asynchronous function takes two memory stores and a data file path as arguments.
    It is designed to upsert (update or insert) data into the memory stores from the data file.

    Args:
        kernel_memory_store (callable): A callable object that represents the kernel memory store where data will be upserted.
        memory_store (callable): A callable object that represents the memory store where data will be upserted.
        data_file_path (str): The path to the data file that contains the data to be upserted.

    Returns:
        None. The function performs an operation that modifies the memory stores in-place.
    """
    with open(file=data_file_path, mode="r", encoding="utf-8") as f:
        data = json.load(f)
        n = 0
        for item in data:
            n += 1
            # check if the item already exists in the memory store
            # if the id doesn't exist, it throws an exception
            try:
                already_created = bool(
                    await memory_store.get(
                        collection_name, item["id"], with_embedding=True
                    )
                )
            except Exception:
                already_created = False
            # if the record doesn't exist, we generate embeddings and save it to the database
            if not already_created:
                await kernel_memory_store.save_information(
                    collection=collection_name,
                    id=item["id"],
                    # the embedding is generated from the text field
                    text=item["content"],
                    description=item["title"],
                )
                print(
                    "Generating embeddings and saving new item:",
                    n,
                    "/",
                    len(data),
                    end="\r",
                )
            else:
                print("Skipping item already exits:", n, "/", len(data), end="\r")

# 5. Add the Chat and Embedding models to the Semantic Kernel

Import the semantic kernel, load the Azure OpenAI key and endpoint, and initialize the semantic kernel.

In [4]:
import semantic_kernel as sk

# get api key and endpoint from .env file
_, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel = sk.Kernel()

Import the needed libraries.

We need the chat completion for having a conversation and text embeddings for generating embeddings.

In [5]:
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextEmbedding,
)

Load the chat deployment name and initialize the chat completions with the required parameters.

In [6]:
# adding azure openai chat service
chat_model_deployment_name = config.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

azure_chat_service = AzureChatCompletion(
    deployment_name=chat_model_deployment_name, endpoint=endpoint, api_key=api_key
)

Add the created chat service to the semantic kernel instance.

In [7]:
kernel.add_chat_service(chat_model_deployment_name, azure_chat_service)
print("Added Azure OpenAI Chat Service...")

Added Azure OpenAI Chat Service...


Load the embeddings deployment name and initialize the text embedding with the required parameters.

In [8]:
# adding azure openai text embedding service
embedding_model_deployment_name = config.get("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME")

azure_text_embedding_service = AzureTextEmbedding(
    deployment_name=embedding_model_deployment_name, endpoint=endpoint, api_key=api_key
)

Add the created embedding service to the semantic kernel instance.

In [9]:
kernel.add_text_embedding_generation_service(
    embedding_model_deployment_name, azure_text_embedding_service
)
print("Added Azure OpenAI Embedding Generation Service...")

Added Azure OpenAI Embedding Generation Service...


# 6. Create or Update Azure Cosmos DB for MongoDB

The semantic kernel can handel the database, collection, index creation.

Import the Azure CosmosDB memory store and initialize it with the parameters defined before.

If the database, collection, and index exist it won't overwrite it.

In [10]:
from semantic_kernel.connectors.memory.azure_cosmosdb import (
    AzureCosmosDBMemoryStore,
)

print("Creating or updating Azure Cosmos DB Memory Store...")
# create azure cosmos db for mongo db vcore api store and collection with vector ivf
# currently, semantic kernel only supports the ivf vector kind
store = await AzureCosmosDBMemoryStore.create(
    database_name=config.get("AZCOSMOS_DATABASE_NAME"),
    collection_name=collection_name,
    index_name=index_name,
    vector_dimensions=vector_dimensions,
    num_lists=num_lists,
    similarity=similarity,
)
print("Finished updating Azure Cosmos DB Memory Store...")

Creating or updating Azure Cosmos DB Memory Store...
Finished updating Azure Cosmos DB Memory Store...


Add the created memory store to the semantic kernel instance.

In [11]:
kernel.register_memory_store(memory_store=store)
print("Registered Azure Cosmos DB Memory Store...")

Registered Azure Cosmos DB Memory Store...


# 7. Generate embeddings and Create Database records

Call the helper function with the JSON data file to generate embeddings and create or update the database records.

If the records already exit it will skip it.

Records are identified by their ids.

The data used here is a dummy data which you can replace with your own.

**Note that you need to specify id, text, and description fields.
The text field is what gets converted to embeddings.**

See the helper function definition for more information.

In [12]:
print("Upserting data to Azure Cosmos DB Memory Store...")
await upsert_data_to_memory_store(kernel.memory, store, "./text-sample.json")

Upserting data to Azure Cosmos DB Memory Store...


# 8. Test the Vector Database

The search function converts the query_term to a vector embedding and finds the similarity between it and the database records.

In [13]:
# each time it calls the embedding model to generate embeddings from your query
query_term = "What is Azure Database for Managed Instances?"
result = await kernel.memory.search(collection_name, query_term)

In [14]:
print(
    f"Result is: {result[0].text}\nRelevance Score: {result[0].relevance}\nFull Record: {result[0].additional_metadata}"
)

Result is: Azure SQL Managed Instance is a fully managed, scalable, and secure SQL Server instance hosted in Azure. It provides features like automatic backups, monitoring, and high availability. SQL Managed Instance supports various data types, such as JSON, spatial, and full-text. You can use Azure SQL Managed Instance to migrate your existing applications, build new applications, and ensure the performance and security of your data. It also integrates with other Azure services, such as Azure App Service and Azure Data Factory.
Relevance Score: 0.8967287909762478
Full Record: {"text": "Azure SQL Managed Instance is a fully managed, scalable, and secure SQL Server instance hosted in Azure. It provides features like automatic backups, monitoring, and high availability. SQL Managed Instance supports various data types, such as JSON, spatial, and full-text. You can use Azure SQL Managed Instance to migrate your existing applications, build new applications, and ensure the performance and

# 9. Create chat function with Azure OpenAI chat model

In [15]:
prompt = """
    You are a chatbot that can have a conversations about any topic related to the provided context.
    Start by saying how relevant the question is using the provided context relevancy score.
    Give explicit answers from the provided context or say 'I don't know' if it does not have an answer.
    provided context: {{$db_record}}

    User: {{$query_term}}
    Chatbot:"""

In [16]:
chat_function = kernel.create_semantic_function(
    prompt, max_tokens=500, temperature=0.0, top_p=0.5
)
context = kernel.create_new_context()

In [17]:
context["query_term"] = query_term
context["db_record"] = result[0].additional_metadata
completions_result = await chat_function.invoke(context=context)

In [18]:
print(completions_result)

The provided context is highly relevant to your question. Azure SQL Managed Instance is a fully managed, scalable, and secure SQL Server instance hosted in Azure. It provides features like automatic backups, monitoring, and high availability. It is a database service that allows you to migrate your existing applications, build new applications, and ensure the performance and security of your data.


# 10. Testing the RAG flow 

In [20]:
import time

query_term = ""
while query_term != "exit":
    query_term = input("Enter a query: ")
    result = await kernel.memory.search(collection_name, query_term)
    context["query_term"] = query_term
    context["db_record"] = result[0].additional_metadata
    completions_result = await chat_function.invoke(context=context)
    print(f"Question:\n{query_term}\nResponse:\n{completions_result}\n")
    time.sleep(5)

Question:
What is Azure Functions?
Response:
The question is highly relevant with a score of 1. Azure Functions is a serverless compute service that allows you to run event-driven code without managing the underlying infrastructure. It supports various programming languages and provides features like automatic scaling, triggers, and bindings. You can use it to build microservices, integrate with other Azure services, and process and transform data.

Question:
What is the best places to go hicking?
Response:
I'm sorry, but that question is not related to the provided context about Azure Stack Hub. Do you have any questions about Azure Stack Hub or related topics?

Question:
exit
Response:
Goodbye!

